In [1]:
def day_predictor(trackss):
    import warnings
    warnings.simplefilter(action='ignore')
    
    good_races1 = {}
    good_races2 = {}
    good_races3 = {}
    good_races4 = {}
    good_races5 = {}
    
    races1 = {}
    races2 = {}
    races3 = {}
    races4 = {}
    races5 = {}

    for trackk in trackss:
        from selenium import webdriver
        from selenium.webdriver.common.keys import Keys
        import pandas as pd
        from selenium.webdriver.common.by import By
        from datetime import datetime
        from selenium.common.exceptions import NoSuchElementException
        import numpy as np
        from scipy.stats import mvn
        import scipy.stats
        import numpy as np
        import time
        import json


        #url='https://www.thegreyhoundrecorder.com.au/form-guides/traralgon/fields/227708/'
        url = trackk
        driver = webdriver.Chrome()
        time.sleep(2)
        driver.get(url)

        time.sleep(2)

        races= driver.find_elements(By.CLASS_NAME,'form-guide-field-event')
        date = driver.find_element(By.CLASS_NAME,'form-guide-meeting__heading').text
        res=''
        num=0

        for race in races:
            header = race.find_element(By.CLASS_NAME,'meeting-event__header')
            name= header.find_element(By.CLASS_NAME,'meeting-event__header-race').text
            dist= header.find_element(By.CLASS_NAME,'meeting-event__header-distance').text
            #print(dist)

            tab = race.find_element(By.CLASS_NAME,'form-guide-event__table')
            df= pd.read_html(tab.get_attribute('outerHTML'))[0]

            df = df.reset_index()
            df['index']=df['index']+1
            df = df.rename(columns={'index':'Box','Rug/Greyhound (box)':'DogName'})
            df['RaceId']=name
            df['Distance']=int(dist[:-1])
            df['Trainer']=df['Trainer'].str.split(' ').str[0].str[0]+' '+df['Trainer'].str.split(' ').str[1]
            df['GR']=df['Our $'].str[1:]
        #     df['Unnamed: 13']=df['Unnamed: 13'].str[1:]
        #     df = df.rename(columns={'Unnamed: 13':'SB'})
            #df['TAB']=df['Unnamed: 10'].str[1:]


            df=df.drop(columns=['Form','Gd','Career','Trk/Dist','Best T/D','Rtg','Our $'])
            df['DogName']=df['DogName'].str.lower()
            df=df[df['DogName']!='vacant box']
            df=df[df['Early Speed']!='Scratched']

            if(num==0):
                res=df
                num+=1
            else:
                res=pd.concat([res,df])
        
        res.reset_index(inplace=True,drop=True)

        for i in range(res.shape[0]):
            name = res.iloc[i,1]
            if('(box' in name):
                res.iloc[i,0]=int(name[-2])
                res.iloc[i,1]=name.split('(')[0]
        #print(res.tail(20))











        ##
        #trk='shepparton'
        trk=url.split('/')[-4].replace('-',' ')
        
        if trk == 'murray bridge straight':
            trk = 'murray bridge'

        races=pd.read_csv('allRaces1.csv')
        #races=races.drop(columns='Unnamed: 0')
        races['date']=pd.to_datetime(races['date'],errors='coerce')
        races['Place'] = pd.to_numeric(races['Place'], errors='coerce')
        races = races.dropna(subset=['Place','Track'])
        races['Place'] = races['Place'].astype(int)
        races['Box']=races['Box'].astype(int)
        races['PIR']=races['PIR'].astype(str)
        races['TrainerName']=races['TrainerFName']+' '+races['TrainerLName']
        races=races.drop(columns=['TrainerLName','TrainerFName'])


        #races = races[races['date']<'2024-01-15']



        a = races[races['Track']=='hobart']
        a = a[a['Distance']==278]
        ls = list(set(a['RaceId']))

        for p in ls:
            races = races[races['RaceId']!=p]

        a = races[races['Track']=='hobart']
        a = a[a['Distance']==515]
        ls = list(set(a['RaceId']))

        for p in ls:
            races = races[races['RaceId']!=p]



        omitted=[]
        dogs_done = set()
        dog_races = {}

        for i in range(0,res.shape[0]):

            name = res.iloc[i,1]
            if(name not in dogs_done):
                dog_past_race = races[races['DogName']==name]
                dog_races[name]=dog_past_race
                dogs_done.add(name)

            if(i%20==0):
                pass
                #print(i)



        trainers_done = set()
        trainer_races = {}

        for i in range(0,res.shape[0]):
            name = res.iloc[i,2]

            if(name not in trainers_done):
                dog_past_race = races[races['TrainerName']==name]
                trainer_races[name]=dog_past_race
                trainers_done.add(name)

            if(i%20==0):
                pass
                #print(i)



        '''unique_track_dist={}
        tracks = races['Track'].unique()
        for i in range(tracks.size):
            dic={}
            track=tracks[i]
            df=races[races['Track']==track]
            dists = df['Distance'].unique()
            for i in dists:
                df2=df[df['Distance']==i]
                ls=[df2['Split'].mean(),df2['Split'].std(),df2['RunTime'].mean(),df2['RunTime'].std()]
                dic[i]=ls
            unique_track_dist[track]=dic
        unique_track_dist['casino'][300]'''

        with open('parameters.json','r') as json_file:
            unique_track_dist = json.load(json_file)



        def pir_function(pirs,places):
            start=0
            mid=0
            fin=0
            for i in range(len(pirs)):
                pir = pirs[i]

                if(pd.isna(pir) or pir=='nan'):
                    start+=4.5
                    mid+=4.5
                    fin+=4.5
                else: 
                    try:
                        if('.' in pir):
                            pir = str(int(float(pir)))


                        if(len(pir)==1):
                            start+=int(pir)-places[i]
                            mid+=int(pir)-places[i]
                            fin+=int(pir)-places[i]
                        elif(len(pir)==2):
                            start+=int(pir[0])-places[i]
                            mid+=int(pir[1])-places[i]
                            fin+=int(pir[1])-places[i]
                        elif(len(pir)==3):
                            start+=int(pir[0])-places[i]
                            mid+=int(pir[1])-places[i]
                            fin+=int(pir[2])-places[i]
                        elif(len(pir)==4):
                            start+=int(pir[0])-places[i]
                            mid+=(int(pir[1])+int(pir[2]))/2-places[i]
                            fin+=int(pir[3])-places[i]
                        elif(len(pir)==5):
                            start+=(int(pir[0])+int(pir[1]))/2-places[i]
                            mid+=int(pir[2])-places[i]
                            fin+=(int(pir[3])+int(pir[4]))/2-places[i]
                        elif(len(pir)==6):
                            start+=(int(pir[0])+int(pir[1]))/2-places[i]
                            mid+=(int(pir[2])+int(pir[3]))/2-places[i]
                            fin+=(int(pir[5])+int(pir[4]))/2-places[i]
                        else:
                            start+=4.5-places[i]
                            mid+=4.5-places[i]
                            fin+=4.5-places[i]
                    except:
                        start+=4.5-places[i]
                        mid+=4.5-places[i]
                        fin+=4.5-places[i]


            start/=len(pirs)
            mid/=len(pirs)
            fin/=len(pirs)
            return([start,mid,fin])




        dogs = list(dog_races.keys())
        lane_prefs={}
        for dog in dogs:
            df=dog_races[dog]
            mean_place=df['Place'].mean()

            ins_mean = df[df['Box'].isin([1,2,3])]['Place'].mean() - mean_place
            mid_mean = df[df['Box'].isin([4,5])]['Place'].mean() - mean_place
            out_mean = df[df['Box'].isin([6,7,8])]['Place'].mean() - mean_place

            means=[ins_mean,mid_mean,out_mean]
            means = [0 if pd.isna(mean) else mean for mean in means]
            lowest=np.argmin(means)
            lane_prefs[dog]=means+[lowest]





        final_result={}
        for k in res['RaceId'].unique():

            race=res[res['RaceId']==k]
            race['psplit'] = None
            race_res=[]


            for j in range(race.shape[0]):
                name = race.iloc[j,1]
                past = dog_races[name]
                past = past[past['Track']==trk]
                pred_split=0

                past.reset_index(drop=True, inplace=True)

                for m in range(past.shape[0]):

                    if(pd.isna(past.iloc[m,6])==False):
        #                 print(past.iloc[m,])
                        params = unique_track_dist[past.iloc[m,11]][str(past.iloc[m,9])]
                        pred_split+=((past.iloc[m,6]-params[0])/params[1])
                if(past.shape[0]!=0):
                    pred_split/=past.shape[0]
                race.iloc[j,-1]=pred_split 
        #     print(race)

            race=race.sort_values(by='Box') 
            race = race.reset_index(drop=True)
            race = race.head(8)


            for i in range(0,race.shape[0]):

                row = race.iloc[i,]

                name = row['DogName']
                trainer = row['Trainer']


                df = dog_races[name]
        #         if(rac)

                trainer = trainer_races[row['Trainer']]
                trainer = trainer[trainer['Track']==trk]


                if(df.shape[0]<5):
                    #print(k,name,df.shape[0])
                    omitted.append(k)

                else: 

                    #mean place
                    mean_place = df['Place'].mean()


                    #lane preferences
                    params = lane_prefs[name]

                    means=params[0:3]
                    best_lane=params[3]

                    boxes=[[1,2,3],[4,5],[6,7,8]]
                    lowest = best_lane
                    box = row['Box']

                    if(box<4):
                        box2=0
                    elif(box<6):
                        box2=1
                    else:
                        box2=2

                    mean_in_pos=means[box2]


                    #last races

                    time_z_avg = 0
                    split_z_avg = 0

                    for j in range(df.shape[0]):
                        row2 = df.iloc[j,]

                        params = unique_track_dist[row2['Track']][str(row2['Distance'])]

                        if(pd.isna(row2['Split'])==False):
                            split_z_avg+=((row2['Split']-params[0])/params[1])
                        if(pd.isna(row2['RunTime'])==False):
                            time_z_avg+=((row2['RunTime']-params[2])/params[3])

                    time_z_avg/=df.shape[0]
                    split_z_avg/=df.shape[0]

                    last_5_times=[]

                    last_5 =df.tail(5)
                    for j in range(5):
                        row2 = last_5.iloc[j,]
            #           
                        params = unique_track_dist[row2['Track']][str(row2['Distance'])]

                        if(pd.isna(row2['RunTime'])==False):
                            last_5_times.append((row2['RunTime']-params[2])/params[3])

                    last_5_times_avg=sum(last_5_times)/5
                    in_form=0
                    if(last_5_times_avg<time_z_avg):
                        in_form=1

                    try:
                        track_params = unique_track_dist[trk][str(row['Distance'])]
                    except Exception as e:
                        #('missing td')
                        #print('missing td')
                        #print('missing td')

                        track_params=[0,0,0,0]

                    pred_split = split_z_avg * track_params[1]+track_params[0]



                    at_cas=df[df['Track']==trk]

                    at_cas_perc= sum(df['Track']==trk)/df.shape[0]

                    ctime_z_avg = 0
                    csplit_z_avg=0


                    for j in range(at_cas.shape[0]):
                        row2 = at_cas.iloc[j,]

                        params = unique_track_dist[row2['Track']][str(row2['Distance'])]

                        if(pd.isna(row2['Split'])==False):
                            csplit_z_avg+=((row2['Split']-params[0])/params[1])

                        if(pd.isna(row2['RunTime'])==False):
                            ctime_z_avg+=((row2['RunTime']-params[2])/params[3])
                    if(at_cas.shape[0]!=0):
                        ctime_z_avg/=at_cas.shape[0]
                        csplit_z_avg/=at_cas.shape[0]

                    pred_split2 = csplit_z_avg * track_params[1]+track_params[0]

                    at_cas_dist=at_cas[at_cas['Distance']==row['Distance']]
                    at_cas_dist_time=0
                    at_cas_dist_time_avg=0
                    in_form2=0

                    if(at_cas_dist.shape[0]>0):
                        final_row = at_cas_dist.iloc[-1,]
                        at_cas_dist_time=(final_row['RunTime']-track_params[2])/track_params[3]
                        at_cas_dist_time_avg=(at_cas_dist['RunTime'].mean()-track_params[2])/track_params[3]
                        if(at_cas_dist_time<at_cas_dist_time_avg):
                            in_form2=1




                    if(trainer.shape[0]==0):
                        mean_trainer_place=4.5
                    else:
                        mean_trainer_place=trainer['Place'].mean()

                    trainer = trainer.tail(5)
                    trainer_last_5_time_z = 0


                    for j in range(trainer.shape[0]):
                        row2 = trainer.iloc[j,]
                        params = unique_track_dist[row2['Track']][str(row2['Distance'])]

                        if(pd.isna(row2['RunTime'])==False):
                            trainer_last_5_time_z+=((row2['RunTime']-params[2])/params[3])
                    if(trainer.shape[0]!=0):
                        trainer_last_5_time_z/=trainer.shape[0]


                    #faster dog inside
                    race = race.sort_values(by='Box', ascending=True)
                    race.reset_index(drop=True, inplace=True)
                    index= race.loc[race['Box'] == box].index[0]

                    faster_inside=0
                    psplit=race.iloc[index,-1]


                    if(pd.isna(psplit)==False):        
                        for j in range(index):
                            if(race.iloc[j,-1]<psplit):
                                faster_inside=1
                                break

                    #next to fast dog
                    next_to_faster=0
                    collision=0
                    try:
                        if(box==1):
                            if(race.iloc[1,-1]<psplit):
                                next_to_faster=1

                        elif(box==max(race['Box'])):
                            if(race.iloc[race.shape[0]-2,-1]<psplit):
                                next_to_faster=1

                        else:
                            if(race.iloc[index-1,-1]<psplit or race.iloc[index+1,-1]<psplit):
                                next_to_faster=1




                        #collision

                        if(box==1):
                            if(lane_prefs[race.iloc[1,1]][3]<best_lane):
                                collision=1

                        elif(box==max(race['Box'])):
                            if(lane_prefs[race.iloc[race.shape[0]-2,1]][3]>best_lane):
                                collision=1


                        else:
                            if(lane_prefs[race.iloc[index-1,1]][3]>best_lane or lane_prefs[race.iloc[index+1,1]][3]<best_lane):
                                collision=1
                    except:
                        pass

                    #first_out
                    first_out=sum(race['psplit']<psplit)
                    if(first_out>0):
                        first_out=0
                    elif(first_out==0):
                        first_out=1



                    #PIR
                    pirs = list(df['PIR'])
                    pir_res = pir_function(pirs,list(df['Place']))



                    placing_box_res=[mean_place,mean_in_pos]
                    last_5_res=[time_z_avg, split_z_avg,pred_split] + last_5_times + [last_5_times_avg,in_form]
                    last_2_res=[ctime_z_avg,csplit_z_avg,pred_split2,at_cas_dist_time,at_cas_dist_time_avg,at_cas_perc,in_form2]
                    trainer_res=[mean_trainer_place,trainer_last_5_time_z]
                    random=[faster_inside, next_to_faster,collision,first_out]


                    cols=['Box','DogName','Distance']
                    row3=list(row[cols])
                    row3+=list(df.tail(1)['Weight'])

                    if(pd.isna(row['GR'])):
                        row3+=[5]
                    else:
                        row3+=[row['GR']] 

                    row3+=placing_box_res+last_5_res+last_2_res
                    row3+=trainer_res+random+pir_res
                    race_res.append(row3)



        #             


            colnames = ['Box','DogName','Distance','Weight','StartPrice']
            colnames+=['mean_place','mean_in_race_pos']
            colnames+=['time_z_avg','split_z_avg','pred_split1']
            colnames+=['t1','t2','t3','t4','t5','last_5_avg','in_form']
            colnames+=['ctime_z_avg','csplit_z_avg','pred_split2','at_cas_dist_time','at_cas_dist_time_avg','at_cas_perc','in_form2']
            colnames+=['t_mean_place','t_5_time_z']
            colnames+=['faster_inside','next_to_faster','collision','first_out','pir1','pir2','pir3']
            df_final = pd.DataFrame(race_res,columns=colnames)
            for i in range(1,9):
                name = 'b'+str(i)
                df_final[name]=np.where(df_final['Box']==i,1,0)
            df_final = df_final.drop(columns=['Box'])


            final_result[k]=df_final






        goood_races1 = []
        goood_races2 = []
        goood_races3 = []
        goood_races4 = []
        goood_races5 = []

        raaces1 = []
        raaces2 = []
        raaces3 = []
        raaces4 = []
        raaces5 = []
        
        for q in range(1,6):
            # from pandas import FutureWarning
            import warnings
            from sklearn.linear_model import Lasso, Ridge,LogisticRegression
            from sklearn.ensemble import RandomForestRegressor
            warnings.simplefilter("ignore", category=FutureWarning)

            data=pd.read_csv('class3/'+trk+'.csv')
            #data=data.dropna()
            data=data.drop(columns=['Unnamed: 0'])
            #data = data[data['Date']<'2023-11-21']


            import json
            with open('preds.json','r') as file:
                predictors2=json.load(file)
            predictors=predictors2[trk]


            pd.set_option('display.float_format', lambda x: f'{x:.2f}')
            from sklearn.linear_model import LinearRegression
            import statsmodels.formula.api as smf



            mods={}
            sds={}
            sds1={}

            #data = data[data['Distance']==342]

            for distance in data['Distance'].unique():
                try:
                    if distance not in [516,720,640,729,732,790,722,760,680,600,431,595,717,620,603,622,277,488]:
                        td = trk + ' ' +str(distance)
                        data2 = data[data['Distance']==distance] 

                        #features = predictors[str(distance)]

                        if q == 1:
                            features = ['split_z_avg', 'last_5_avg', 'at_cas_dist_time', 'mean_place', 'pir3', 'at_cas_perc', 'collision', 'i1', 'mean_in_race_pos']

                        if q == 2:
                            features = ['faster_inside','i2','i1','split_z_avg','last_5_avg']
                        
                        if q == 3:
                            features = ['at_cas_dist_time_avg', 'faster_inside', 'i2', 'i1', 'split_z_avg', 'last_5_avg', 'mean_place', 'time_z_avg']
                        
                        if q == 4:
                            features = ['at_cas_dist_time_avg', 'faster_inside', 'i2', 'i1', 'split_z_avg', 'last_5_avg', 'mean_place', 'at_cas_perc', 'in_form', 'mean_in_race_pos', 'last_5_avg']
                        
                        if q == 5:
                            features = ['ctime_z_avg','last_5_avg']


                        #print(len(data2))
                        data2=data2.dropna(subset = features)
                        #print(len(data2))

                        mod = LogisticRegression(random_state=42,max_iter=2000).fit(data2[features], data2['Win'])
                        mods[distance]=mod
                except error as e:
                    pass
                    #print(e)

            
            for race in list(final_result.keys()):
            # for race in ['Race 12','Race 11','Race 10']:

                if race not in omitted:
                # for race in ['Race 12']:
                    info = final_result[race]

                    #info = info[info['Distance']==342]

                    i1 = []
                    i2 = []
                    for x in info.index:
                        if info.loc[x,'b1'] == 1 or info.loc[x,'b2'] == 1 or info.loc[x,'b3'] == 1:
                            i1.append(1)
                            i2.append(0)
                        elif info.loc[x,'b4'] == 1 or info.loc[x,'b5'] or info.loc[x,'b6']== 1:
                            i1.append(0)
                            i2.append(1)
                        else:
                            i1.append(0)
                            i2.append(0)
                    info['i1'] = i1
                    info['i2'] = i2
                #         print(info)
                    if(info.shape[0]<7) and int(info.head(1)['Distance']) not in [516,720,640,488,729,732,277,790,722,760,680,600,431,595,717,620,603,622]:
                        #print(info)
                        d = int(info.head(1)['Distance'])
                        #print(d)
                        td = trk+' '+str(d)
                        #print(race,d)


                        data2 = data[data['Distance']==d] 

                        mod = mods[d]

                        weights = []
                        for x in range(0,info.shape[0]):
                            weights.append(1)
                        info['Weight']=weights

                        s1 = info.shape[0]
                        reason = info.columns[info.isna().any()].tolist()
                        info=info.dropna(subset = features)
                        s2 = info.shape[0]

                        if(s2!=8 and s1!=s2):
                            pass
                            #print('dog ditched',s1,s2)
                            #print('error reason:',reason)

                        #features = predictors[str(d)]
                        if q == 1:
                            features = ['split_z_avg', 'last_5_avg', 'at_cas_dist_time', 'mean_place', 'pir3', 'at_cas_perc', 'collision', 'i1', 'mean_in_race_pos']

                        if q == 2:
                            features = ['faster_inside','i2','i1','split_z_avg','last_5_avg']
                        
                        if q == 3:
                            features = ['at_cas_dist_time_avg', 'faster_inside', 'i2', 'i1', 'split_z_avg', 'last_5_avg', 'mean_place', 'time_z_avg']
                        
                        if q == 4:
                            features = ['at_cas_dist_time_avg', 'faster_inside', 'i2', 'i1', 'split_z_avg', 'last_5_avg', 'mean_place', 'at_cas_perc', 'in_form', 'mean_in_race_pos', 'last_5_avg']
                        
                        if q == 5:
                            features = ['ctime_z_avg','last_5_avg']


                        #print(features)

                        #print(info)
                        preds3= mod.predict_proba(info[features])
                        preds3 = preds3/sum(preds3)
                        preds = 1/np.transpose(preds3)[1]
                        
                        
                        predss = sorted(list(preds))
                        if q == 1 and predss[1]-predss[0] > 1.3:
                            goood_races1.append(int(race.lstrip('Race ')))
                        if q == 2 and predss[1]-predss[0] > 1.3:
                            goood_races2.append(int(race.lstrip('Race ')))
                        if q == 3 and predss[1]-predss[0] > 1.3:
                            goood_races3.append(int(race.lstrip('Race ')))
                        if q == 4 and predss[1]-predss[0] > 1.3:
                            goood_races4.append(int(race.lstrip('Race ')))
                        if q == 5 and predss[1]-predss[0] > 1.3:
                            goood_races5.append(int(race.lstrip('Race ')))
                            
                            
                            

                        output=pd.DataFrame(preds)

                        output['name']=info['DogName']


                        output['pred_prices']=preds
                        #output['lower']=0.7*preds
                        #output['upper']=0.9*preds



                        #output['prices']=info['StartPrice']

                        output['pred_prices']=output['pred_prices'].round(2)
                        output=output.sort_values(by='pred_prices')
                #                     output['top2']=place1(list(preds),sds1[d])
                #                     output['top3']=place(list(preds),sds1[d])
#                         print(output)
#                         print()
#                         print('------------------------------------------------------------')
                        if q == 1:
                            raaces1.append(race+': '+str(d)+'m')
                            raaces1.append(output)
                        if q == 2:
                            raaces2.append(race+': '+str(d)+'m')
                            raaces2.append(output)
                        if q == 3:
                            raaces3.append(race+': '+str(d)+'m')
                            raaces3.append(output)
                        if q == 4:
                            raaces4.append(race+': '+str(d)+'m')
                            raaces4.append(output)
                        if q == 5:
                            raaces5.append(race+': '+str(d)+'m')
                            raaces5.append(output)
                            
            
        races1[trk] = raaces1
        races2[trk] = raaces2
        races3[trk] = raaces3
        races4[trk] = raaces4
        races5[trk] = raaces5

        good_races1[trk] = goood_races1
        good_races2[trk] = goood_races2
        good_races3[trk] = goood_races3
        good_races4[trk] = goood_races4
        good_races5[trk] = goood_races5
            
    return [races1,races2,races3,races4,races5,good_races1,good_races2,good_races3,good_races4,good_races5]

In [2]:
ls = day_predictor(['https://www.thegreyhoundrecorder.com.au/form-guides/murray-bridge-straight/fields/228304/'])

races1 = ls[0]
races2 = ls[1]
races3 = ls[2]
races4 = ls[3]
races5 = ls[4]

good_races1 = ls[5]
good_races2 = ls[6]
good_races3 = ls[7]
good_races4 = ls[8]
good_races5 = ls[9]

In [3]:
track = 'murray bridge'

In [6]:
print(good_races1[track])
print(good_races2[track])
print(good_races3[track])
print(good_races4[track])
print(good_races5[track])

[2, 3, 5, 6, 10]
[3, 6]
[2, 3, 5, 6, 10]
[2, 3, 5, 6, 10]
[3, 4, 5, 6, 10]


In [7]:
races1[track]

['Race 2: 300m',
       0           name  pred_prices
 3  2.86  woodside star         2.86
 1  4.38   give me room         4.38
 5  5.59   take a shine         5.59
 4 11.35   searing lass        11.35
 2 12.67      tree town        12.67
 0 13.15        beasley        13.15,
 'Race 3: 350m',
      0            name  pred_prices
 1 2.02  oakvale outlaw         2.02
 0 5.18     aston radar         5.18
 2 5.85  drive the jack         5.85
 3 7.16   razzle darryl         7.16,
 'Race 4: 350m',
       0             name  pred_prices
 1  2.72    current model         2.72
 0  3.29    pink confetti         3.29
 5  5.68           nikita         5.68
 2 17.60  all about larry        17.60
 4 20.08        go jet go        20.08
 3 22.17  terrific janine        22.17,
 'Race 5: 300m',
      0              name  pred_prices
 2 3.71    not for profit         3.71
 4 5.91        go go snow         5.91
 0 6.35         my reason         6.35
 3 6.64   woodside gracie         6.64
 1 7.51       com

In [7]:
#final_result['Race 4'].to_csv('e.csv')

In [8]:
#final_result['Race 4'][features]

In [9]:
# import pandas as pd
# df = pd.read_csv('allRaces1.csv')
# df = df[df['Track']=='capalaba']
# df.tail(60)

In [10]:
# df[df['RaceId']==996378324]